In [1]:
import torch

# data/model/loss/optimize/init/train

# data

# model

In [2]:
##encoder 2 decoer

In [5]:
torch.nn.Embedding?

In [46]:
torch.nn.GRU?

In [8]:
torch.permute?

In [17]:
torch.squeeze?

In [44]:
class Encoder(torch.nn.Module):
    def __init__(self,num_token,dim_embed,num_hidden,dim_hidden):
        super(Encoder,self).__init__()
        ##embed
        self.embedding = torch.nn.Embedding(num_embeddings=num_token,embedding_dim=dim_embed)
        self.gru = torch.nn.GRU(input_size = dim_embed,hidden_size=dim_hidden,num_layers=num_hidden
                                ,bias = True,bidirectional = True)
    def forward(self, x):
        ##non batch first
        ##batch_size,seq_len,dim -> seq_len,batch_size,dim
        x = self.embedding(x)
        x = torch.permute(x,[1,0,2])
        output,h_n = self.gru(x)
        #h_n:seq_len,batch_size,num_directions,dim
        print(output.shape)
        print(h_n.shape)
        #h_n = h_n[-1*2]
        return output,h_n

In [45]:
encoder = Encoder(num_token =10, dim_embed=8, dim_hidden=16,
                         num_hidden=2)
encoder.eval()
X = torch.zeros((4, 7), dtype=torch.long)
output, state = encoder(X)
output.shape,state.shape

torch.Size([7, 4, 32])
torch.Size([4, 4, 16])


(torch.Size([7, 4, 32]), torch.Size([4, 4, 16]))

In [ ]:
class Decoder(torch.nn.Module):
    def __init__(self,num_token,dim_embed,num_hidden,dim_hidden):
        super(Encoder,self).__init__()
        ##embed
        self.embedding = torch.nn.Embedding(num_embeddings=num_token,embedding_dim=dim_embed)
        self.gru = torch.nn.GRU(input_size = dim_embed,hidden_size=dim_hidden,bias = True,bidirectional = False)
        self.dense = torch.nn.Linear(input_size=num_hidden,out_features=num_token)
    
    def init_state(self,encoding_out):
        return encoding_out[-1]
    
    def forward(self,x,state):
        ##x:batch_size,seq_len,dim
        ##state:batch_size,num_layers,dim
        x = torch.permute(x,[1,0,2])
        state = torch.permute(state,[1,0,2])
        state = self.init(state)
        output,state = self.gru(x,state)